# 前回の内容

* 辞書

# 今回の内容

* テストとデバッグ

# 雨漏りのアナロジー

屋根が雨漏りをしていて、ポタポタと雨水が頭上に落ちてくるとしよう...

* 頭を触って、濡れているかどうかを確認する
  * テスト（濡れているのか？そして、この濡れは許容範囲なのか？？）
* 傘をさす
  * 防御的プログラミング/セキュアプログラミング（雨水が落ちてきてきたときに対処ができてればいいじゃん）
* 雨漏りをなおす
  * バグを除去する/デバッグ（そもそも雨水落ちてくるのが悪いじゃん）

# テスト、防御的プログラミング、デバッグ

* 防御的プログラミング
  * 関数の仕様を記述する
  * プログラムをモジュール化する
  * 入出力の条件をチェックする (assertion)

* テスト
  * 入出力を関数仕様と比較する
  * 動かない場合を見つける
  * どうやったら自分のプログラムを壊せるか、動作を変にできるか、を考える

* デバッグ
  * エラーをもたらしている原因となるイベントを調べる
  * なぜ動かないのかを考える
  * どうやったら治せるかを考える


# テスト

まずはテストから

# テスト・デバッグに備えよ（コーディングを）

* コード設計をテストを考慮したものにする考慮したものにする
* プログラムをモジュール化して、テストやデバッグを個別にできるようにするようにする
* モジュールの制約を記述する
  * 入力として想定しているものは
  * 出力として想定しているものは
* コード設計の背景にある**仮定**を記述する


# テストはいつすべきか

* 少なくともコードが動くとき... （インタプリタが出力するエラーを消す）
  * 構文エラーをなくして
  * 静的意味論エラーをなくして

* いくつかの**期待する**出力が手元にあるとき
  * 入力のセット
  * その入力に対して期待される出力のセット
  

# テストの段階

* 単体テスト（unit test）
  * プログラムにおいて個々の**部分**に対する正当性の確認
  * 関数などの単体が正常に動いているか設計されたテスト

* 回帰テスト（regression test）
  * プログラムを改変したときに副作用的に現れるバグに対するテスト
  * 部分的な変更に対して他の要素についてもテストを実施する
  * 多くの場合はテストを自動化することによって実施する（Continuous Integration:CI, Continuous Delivery:CD）
* 統合テスト（integration test）
  * プログラムが全体として意図通りに動いているかを確認するテスト
  * 単体テストが通っていても統合テストが通らない場合は、単体テストのケースに抜けがある

# テスト：いくつかのアプローチ

テストとは、バグを見つけ出す可能性の高い入力の集合（テストスイート）を作り上げること
* 集合の分割
  * 入力の集合をいくつかの部分集合に分ける
* 経路
  * プログラム実行時の経路を発見するする

## 方法

* 直感的にバウンダリ（境界）を求めて、境界テストを行う
```
def is_bigger(x, y):
  """ x, yはintと仮定
  xがyよりも大きければTrueを返す そうでなければFalseを返す """
```
* ランダムテスト
  * 確率論的に、コードの正しさを実証
  * テストするだけ正しさの度合いも上がるだろうという考え方

* ブラックボックステスト
  * 仕様から経路を調べる発見的方法
* グラスボックステスト
  * コードを通して各経路を調べる発見的手法

# ブラックボックステスト

```
def sqrt(x, e):
  """ x, yはfloatと仮定 xは0以上、eは0より大きいと仮定
  ans*ansがx-e以上かつx+e以下となるような変数ansを返す"""
```

* コードを見ずにテストをする
* バイアスを避けるために実装をした者以外の人がテストするとよい
* テストは実装が変わったとしても再利用できる
* 仕様から経路を調べる
 * テストケースを異なる自然な部分集合から作る
 * 境界条件を考慮する（空のリストや1つの要素からなるリスト、大きな数、小さな数）


# ブラックボックステストの例
```
def sqrt(x, e):
  """ x, yはfloatと仮定 xは0以上、eは0より大きいと仮定
  ans*ansがx-e以上かつx+e以下となるような変数ansを返す"""
```

|  ケース  | x | e|
| ---- | ---- | ---- |
|  境界  |  0  | 0.0001|
|  1より小さい  |  0.01  |0.0001|
|  無理数 |  2  |0.0001|
|  極端な条件  |  2  |sys.float_info.min|
|  極端な条件  |  sys.float_info.min |sys.float_info.min|
|  極端な条件  |  sys.float_info.max |sys.float_info.min|
|  極端な条件  |  sys.float_info.min  |sys.float_info.max|
|  極端な条件  |  sys.float_info.max  |sys.float_info.max|


# グラスボックステスト
```
def abs(x):
""" 入力xはintと仮定
xが0以上ならxを返す。そうでなければ-xを返す """ 
  if x < -1:
    return –x
  else:
    return x
```
* コードを直接テストケースの設計に使う
*　可能性のある経路全てをテストするとき、**経路完全**と呼ぶ
* 経路完全を達成するのは難しい
  * ループの回数や再帰の深さにも依存
  * 入力によって異なる経路をとる
* ガイドライン
  * 分岐：すべてのif文について両方の分岐を試す
  * except節：忘れずに実行する（try-except構文）
  * forループ：
    * ループをパスする場合
    * ループが1回だけ実行される場合
    * ループが2回以上実行される場合
  * whileループ：
    * whileループから脱出する（break）条件について全てテストする
  * 再帰
    * 再帰しない場合
    * 1回再帰する場合
    * 2回以上再帰する場合

# グラスボックステストの例
```
def abs(x):
""" 入力xはintと仮定
xが0以上ならxを返す。そうでなければ-xを返す """ 
  if x < -1:
    return –x
  else:
    return x
```

* 経路完全テストケース: 例えば{2, −2}
* abs(-1)はなにを返すか
  * -1
* 経路完全だからといってバグを見つけ出すことが保証されているわけではない
* 境界ケースでのテストが必要


# デバッグ

* 学習によって向上する能力
* 目的
  * バグのないプログラムを得ること
* ツール（デバッガ）
  * IDE組み込みのツール
  * https://pythontutor.com のような可視化ツール
  * **print**文
  * 頭を使って頑張る（システマチックに管理！）

# printデバッグ

* （頭の中にある）仮説をテストするには良い方法
* どこにprintを置くか
  * 関数が呼び出されたらすぐ
  * 変数が変更されるだろうとき
  * 関数が結果を返したら

* ここでも二分法が使える
  * printを置くことでバグの所在を絞り込む
  * どの値がバグを引き起こしているのかを調べる

In [ ]:
def isPal(x):
    """xはリスト リストが回文ならTrue、そうでなければFalseを返す"""
    temp = x
    temp.reverse 
    if temp == x:
        return True
    else:
        return False

def silly(n): 
    """nは正のint n個の入力を受け取り、
    入力文字列が回文なら'Yes'、そうでなければ'No'を出力"""
    for i in range(n):
        result = []
        elem = input('要素を入力: ')
        result.append(elem)
        print(result)
        if isPal(result):
            print('Yes')
        else:
            print('No')

silly(2)

In [ ]:
def isPal(x):
    """xはリスト リストが回文ならTrue、そうでなければFalseを返す"""
    temp = x[:]
    temp.reverse()
    if temp == x:
        return True
    else:
        return False

def silly(n): 
    """nは正のint n個の入力を受け取り、
    入力文字列が回文なら'Yes'、そうでなければ'No'を出力"""
    result = []
    for i in range(n):
        elem = input('要素を入力: ')
        result.append(elem)
        print(result)
        if isPal(result):
            print('Yes')
        else:
            print('No')

silly(2)

# デバッグの心得

* まずコードを調べよ
  * **なにが間違っているか**を調べるのはやめよう
  * どうやったら予期しない結果を引き起こすことができるかをクリエイティブに考える！
* 科学的な方法を試みよ
  * 利用可能なデータを調査
  * 仮説を立てて考える
  * 繰り返し試みる
  * とりあえず単純な入力で考える
* やりがちなことをしてないかチェック
  * 関数への引数の順序
  * 変数の初期化
  * カッコ忘れ
  * float型のチェックに==をつかってないか
  * 副作用の忘れ
  * エイリアシング
* 誰かに頼れ
* 書いてあることを信じるな
* ちょっと離れてドキュメンテーションでもやってみよ
* 寝ろ

# エラーメッセージをとにかく見て！

エラーメッセージに答えが書かれていることが多い。とにかくエラーメッセージに注目！

In [ ]:
hoge = [1,2,3,4,5]
hoge[5]

In [ ]:
float(hoge)

In [ ]:
aaaa

In [ ]:
'123'/123

In [ ]:
a = len([1,2,3,4,5]
print(a)

# ロジックのエラーへの対処は難しい

* コードを書く前にちょっと考える
* 図を書いて考えてみる
* 他の人に話してみる


# やってはいけない
* プログラム全体を書く
* テストを全体に対して行う
* 全体をデバッグする

# 推奨
* 関数に分けて書く
  * 関数をテストする
  * 関数をデバッグする
* 関数に分けて書く
  * 関数をテストする
  * 関数をデバッグする
* 関数に分けて書く
  * 関数をテストする
  * 関数をデバッグする
* このプロセスを繰り返して全体のプログラムを構成する

# やってはいけない（コード変更編）
1. コードを変更する
2. どこにバグがあったのか覚える
3. コードをテストする
4. どこにバグがあったのか忘れる
5. そもそもどのコードを変更したのか忘れる
6. わけがわからないよ...

#　推奨（コード変更編）
1. コードをバックアップする
2. コードを変更する
3. コメントに潜在的なバグについて書いておく
4. コードをテストする
5. 古いバージョン（バックアップ）と新しいコードを比較する

# 例外とアサーション
* プログラムが予期しない状態に陥ったらどうなるか
  * 例外が起きる

```
test = [1,2,3]
test[3]
```

* いろんな例外
  * SyntaxError: プログラムを正常にパースできなかったとき
  * NameError: 変数が見つからなかったとき
  * TypeError: 正しい型でなかったとき
  * ValueError: 値が異常だったとき
  * IOError: 入出力が異常だったとき（ファイルが見つからないなど）
  * AttributeError: 属性が見つからなかったとき

In [ ]:
test = [1,2,3]
test[3]

# 例外の処理

* Pythonでは例外を扱うためのハンドラが備わっている
  * try-exceptブロック

```
try:
  a = int(input("数を入力せよ"))
  b = int(input("数を入力せよ"))
  print(a/b)
except:
  print("何かがおかしい！")
```

* tryブロック内で例外が起こるとexceptブロック内が呼び出される（処理される）

In [ ]:
try:
  a = int(input("数を入力せよ"))
  b = int(input("数を入力せよ"))
  print(a/b)
except:
  print("何かがおかしい！")

# 特定の例外の処理

```
try:
  a = int(input("数を入力せよ"))
  b = int(input("数を入力せよ"))
  print(a/b)
except ValueError:
  print("値がおかしいよ！")
except ZeroDivisionError:
  print("ゼロで割るなんておかしいよ！")
except:
  print("何かがおかしい！")
```

In [ ]:
try:
  a = int(input("数を入力せよ"))
  b = int(input("数を入力せよ"))
  print(a/b)
except ValueError:
  print("値がおかしいよ！")
except ZeroDivisionError:
  print("ゼロで割るなんておかしいよ！")
except:
  print("何かがおかしい！")

In [ ]:
try:
  a = int(input("数を入力せよ"))
  b = int(input("数を入力せよ"))
  print(a/b)
except ValueError:
  print("値がおかしいよ！")
except ZeroDivisionError:
  print("ゼロで割るなんておかしいよ！")
except:
  print("何かがおかしい！")
else:
  print("おかしくなかったよ")
finally:
  print("いずれにしてもこれはでるよ")
  #ファイルを閉じたり後始末するのに使う

# 例外はどのように処理をすべきか

1. なにもしない
  * デフォルトの値を代入して処理を続けるなど...
  * ユーザがフィードバックを得られない難点がある
2. エラーを返す
  * エラーがあったことがフィードバックできる
  * どんな値を返すべきかは臨機応変に
  * 割と考えなくてはならない
3. 実行を止めてエラーを出す
  * 致命的なエラーの場合は実行を止めたほうがよい
  * どんなエラーだったのかフィードバックする
  * `raise Exception("説明")`

In [ ]:
raise Exception("こんな説明です")

In [ ]:
# 例外の例
def getRatios(vect1, vect2):
    """vect1とvect2を同じ長さのリストとする。
    vet1[i]/vect2[i]を意味する値からなるリストを返す。"""
    ratios = []
    for index in range(len(vect1)):
        try:
            ratios.append(vect1[index] / float(vect2[index]))
        except ZeroDivisionError:
            ratios.append(float('nan')) # nan = Not a Number
        except:
            raise ValueError('getRatiosは不適切な引数で呼び出されました')
    return ratios

# 例外の例

* 氏名と複数の科目の点数が記録されているリストがある
* 上のリストについて平均点を加えた新しいリストを作ることにする

```
test_scores = [[["松村","耕平"],[80.0,70.0,84.0]], [["河野","隼一郎"],[90.0,100.0,73.0]]
```

In [ ]:
test_scores = [[["松村","耕平"],[80.0,70.0,84.0]], [["河野","隼一郎"],[90.0,100.0,73.0]]]

def get_stats(score_list):
  new_list = []
  for e in score_list:
    new_list.append([e[0], e[1], avg(e[1])])
  return new_list
def avg(scores):
    return sum(scores)/len(scores)

get_stats(test_scores)

In [ ]:
test_scores = [ [["松村","耕平"],[80.0,70.0,84.0]], 
                [["河野","隼一郎"],[90.0,100.0,73.0]],
                [["バグ","太朗"],[]]]

get_stats(test_scores)

# 例外方略１：エラーがあることを表示する

```
def avg(scores):
    try:
        return sum(scores)/len(scores)
    except ZeroDivisionEroor:
        print("警告：点数が記録されていません")
```


In [ ]:
def avg(scores):
    try:
        return sum(scores)/len(scores)
    except ZeroDivisionError:
        print("警告：点数が記録されていません")

# 例外方略2：点数が登録されていない場合0点として扱う

```
def avg(scores):
    try:
        return sum(scores)/len(scores)
    except ZeroDivisionError:
        print("警告：点数が記録されていません")
        return 0.0
```


In [ ]:
def avg(scores):
    try:
        return sum(scores)/len(scores)
    except ZeroDivisionError:
        print("警告：点数が記録されていません")
        return 0.0

# アサーション

* 仮定したものが、仮定通りであるかを確認したい
* 計算結果が期待したどおりかを確認したい
* **assert**文を使うと、条件外の場合はAssertionError例外を起こすことができる
* 防御的プログラミング

```
def avg(scores):
    assert len(scores) !=0, "点数が記録されていません"
    return sum(scores)/len(scores)
```

* 上の例だと、scoresリストが空(=len(scores)が0)だったらAssertionError例外を起こす
* そうでなければスルー

In [ ]:
def avg(scores):
    assert len(scores) !=0, "点数が記録されていません"
    return sum(scores)/len(scores)

# 防御的プログラミングとしてのアサーション
* アサーションは予期しない状況の場合にプログラマに対応を投げない
* 期待される条件でなかった場合は実行の中断が保証される
* 入力のチェックによく使われる
* 出力のチェックにもよく使われる
* バグの場所を特定するのにも便利である（デバッグツールとして）